# PTF_MVT 02: AZ Processor Testing

**Purpose**: Test complete AZ processor (Bronze → Silver)

**Tests**: Filters, capitals, movements, exposures, coassurance, transforms

---

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))
print(f"Project root: {project_root}")

In [ ]:
from pyspark.sql import SparkSession
from azfr_fsspec_utils import fspath
import azfr_fsspec_abfs

azfr_fsspec_abfs.use()

spark = SparkSession.builder \
    .appName("PTF_MVT_AZ_Processor") \
    .getOrCreate()

print(f"✓ Spark {spark.version}")

## 1. Initialize

In [ ]:
from utils.loaders.config_loader import ConfigLoader
from utils.logger import PipelineLogger

config = ConfigLoader(str(project_root / "config" / "config.yml"))
logger = PipelineLogger("az_test")

VISION = "202509"
print(f"Vision: {VISION}")

## 2. Run AZ Processor

In [ ]:
from src.processors.ptf_mvt_processors import AZProcessor

try:
    az_processor = AZProcessor(spark, config, logger)
    df_az = az_processor.run(VISION)
    
    if df_az is not None:
        print(f"✓ AZ Processor: {df_az.count():,} rows")
        print(f"  Columns: {len(df_az.columns)}")
    else:
        print("⚠ AZ Processor returned None")
except Exception as e:
    print(f"✗ Error: {e}")
    df_az = None

## 3. Verify Key Columns

In [ ]:
if df_az is not None:
    # Capitals
    df_az.select('nopol', 'smp_100', 'lci_100', 'perte_exp', 'risque_direct').show(3)

    # Coassurance
    df_az.select('nopol', 'top_coass', 'coass', 'partcie').show(3)    

    # Movements
    df_az.select('nopol', 'nbafn', 'nbres', 'nbrpt', 'nbrpc', 'nbptf').show(3)


## Summary

In [ ]:
print("="*50)
print("AZ PROCESSOR TESTING COMPLETE")
print("="*50)
print(f"Result: {'✓ Success' if df_az is not None else '✗ Failed'}")
print("→ Next: Notebook 03 - AZEC Processor")